In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('../data/PRSA_data_2010.1.1-2014.12.31.csv')

# Show first few rows (Verification of succesfull imports)
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,cbwd,Iws,Is,Ir
0,1,2010,1,1,0,NaN,-21,-11.0,1021.0,NW,1.79,0,0
1,2,2010,1,1,1,NaN,-21,-12.0,1020.0,NW,4.92,0,0
2,3,2010,1,1,2,NaN,-21,-11.0,1019.0,NW,6.71,0,0
3,4,2010,1,1,3,NaN,-21,-14.0,1019.0,NW,9.84,0,0
4,5,2010,1,1,4,NaN,-20,-12.0,1018.0,NW,12.97,0,0


In [9]:
# Overvieew of the dataset

# Check the shape (rows, columns)
print(f"Dataset shape: {df.shape}")

# Check column info and missing values
df.info()

# Summary statistics for numerical columns
df.describe()

Dataset shape: (43824, 13)
<class 'pandas.DataFrame'>
RangeIndex: 43824 entries, 0 to 43823
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      43824 non-null  int64  
 1   year    43824 non-null  int64  
 2   month   43824 non-null  int64  
 3   day     43824 non-null  int64  
 4   hour    43824 non-null  int64  
 5   pm2.5   41757 non-null  float64
 6   DEWP    43824 non-null  int64  
 7   TEMP    43824 non-null  float64
 8   PRES    43824 non-null  float64
 9   cbwd    43824 non-null  str    
 10  Iws     43824 non-null  float64
 11  Is      43824 non-null  int64  
 12  Ir      43824 non-null  int64  
dtypes: float64(4), int64(8), str(1)
memory usage: 4.3 MB


,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir
count,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,41757.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000,43824.000000
mean,21912.500000,2012.000000,6.523549,15.727820,11.500000,98.613215,1.817246,12.448521,1016.447654,23.889140,0.052734,0.194916
std,12651.043435,1.413842,3.448572,8.799425,6.922266,92.050387,14.433440,12.198613,10.268698,50.010635,0.760375,1.415867
min,1.000000,2010.000000,1.000000,1.000000,0.000000,0.000000,-40.000000,-19.000000,991.000000,0.450000,0.000000,0.000000
25%,10956.750000,2011.000000,4.000000,8.000000,5.750000,29.000000,-10.000000,2.000000,1008.000000,1.790000,0.000000,0.000000
50%,21912.500000,2012.000000,7.000000,16.000000,11.500000,72.000000,2.000000,14.000000,1016.000000,5.370000,0.000000,0.000000
75%,32868.250000,2013.000000,10.000000,23.000000,17.250000,137.000000,15.000000,23.000000,1025.000000,21.910000,0.000000,0.000000
max,43824.000000,2014.000000,12.000000,31.000000,23.000000,994.000000,28.000000,42.000000,1046.000000,585.600000,27.000000,36.000000


In [11]:
# pm2.5 has 4067 missing values. To handle the missing values we will dropt the rows with missing values because the missing percentage is relatively small (9.3%)

# Check how many missing before dropping 
print(f"Rows before dropping: {len(df)}")

# Drop the rows where pm2.5 is NaN
df = df.dropna(subset=['pm2.5']).copy()

# Check the new shape
print(f"Rows after dropping: {len(df)}")
print(f"Missing pm2.5 now: {df['pm2.5'].isnull().sum()}")

Rows before dropping: 43824
Rows after dropping: 41757
Missing pm2.5 now: 0


In [12]:
# Create datetime column from year, month, day, hour
df['datetime'] = pd.to_datetime(df[['year','month','day','hour']])

# Sort by datetime
df = df.sort_values('datetime').reset_index(drop=True)

# Check the result - show first few rows of datetime and pm2.5
df[['datetime','pm2.5','TEMP']].head()

,datetime,pm2.5,TEMP
0,2010-01-02 00:00:00,129.0,-4.0
1,2010-01-02 01:00:00,148.0,-4.0
2,2010-01-02 02:00:00,159.0,-5.0
3,2010-01-02 03:00:00,181.0,-5.0
4,2010-01-02 04:00:00,138.0,-5.0


In [13]:
# Extract time features
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month

# Create season: 1=winter (Dec, Jan, Feb), 2=spring (Mar, Apr, May), 
# 3=summer (Jun, Jul, Aug), 4=fall (Sep, Oct, Nov)

df['season'] = (df['month']%12+3)//3

# One-hot encode wind direction
df = pd.get_dummies(df, columns=['cbwd'], prefix='wind')

# Check the new columns
df.head()

,No,year,month,day,hour,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,datetime,season,wind_NE,wind_NW,wind_SE,wind_cv
0,25,2010,1,2,0,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1,False,False,True,False
1,26,2010,1,2,1,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1,False,False,True,False
2,27,2010,1,2,2,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1,False,False,True,False
3,28,2010,1,2,3,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1,False,False,True,False
4,29,2010,1,2,4,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1,False,False,True,False


In [14]:
# Create lag features (shift pm2.5 backward)
df['lag_1h'] = df['pm2.5'].shift(1)
df['lag_24h'] = df['pm2.5'].shift(24)

# Check the first few rows to see the new columns
df[['datetime', 'pm2.5', 'lag_1h', 'lag_24h']].head(25)

,datetime,pm2.5,lag_1h,lag_24h
0,2010-01-02 00:00:00,129.0,NaN,NaN
1,2010-01-02 01:00:00,148.0,129.0,NaN
2,2010-01-02 02:00:00,159.0,148.0,NaN
3,2010-01-02 03:00:00,181.0,159.0,NaN
4,2010-01-02 04:00:00,138.0,181.0,NaN
5,2010-01-02 05:00:00,109.0,138.0,NaN
6,2010-01-02 06:00:00,105.0,109.0,NaN
7,2010-01-02 07:00:00,124.0,105.0,NaN
8,2010-01-02 08:00:00,120.0,124.0,NaN
9,2010-01-02 09:00:00,132.0,120.0,NaN


In [15]:
# Check how many rows before dropping
print(f"Rows before dropping lag NaNs: {len(df)}")

# Drop rows where either lag column is NaN
df = df.dropna(subset=['lag_1h', 'lag_24h']).copy()

print(f"Rows after dropping: {len(df)}")

Rows before dropping lag NaNs: 41757
Rows after dropping: 41733


In [16]:
# Define feature columns
feature_cols = ['TEMP', 'PRES', 'DEWP', 'Iws', 'Is', 'Ir', 
                'hour', 'month', 'season', 'lag_1h', 'lag_24h'] + \
                [col for col in df.columns if col.startswith('wind_')]

X = df[feature_cols]
y = df['pm2.5']

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("\nFirst few feature names:")
print(X.columns.tolist()[:10])  # show first 10

Features shape: (41733, 15)
Target shape: (41733,)

First few feature names:
['TEMP', 'PRES', 'DEWP', 'Iws', 'Is', 'Ir', 'hour', 'month', 'season', 'lag_1h']


In [17]:
split_date = '2013-01-01'
train_mask = df['datetime'] < split_date
test_mask = df['datetime'] >= split_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

Training samples: 24394
Test samples: 17339


In [18]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [19]:
# Initialize scaler
scaler = StandardScaler()

# Fit on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Create and train the model
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Predict on test set
y_pred_lr = lr.predict(X_test_scaled)

In [21]:
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred_lr)
rmse = mean_squared_error(y_test, y_pred_lr, squared=False)
r2 = r2_score(y_test, y_pred_lr)

print("Linear Regression Performance:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

TypeError: got an unexpected keyword argument 'squared'